In [2]:
import pandas as pd
import os
from pybaseball import statcast, pitching_stats, batting_stats, playerid_lookup, statcast_batter, statcast_pitcher

from pull_data import *
%load_ext autoreload
%autoreload 2


In [3]:
from stat_miner import *

In [4]:
from rotowire import get_game_lineups

In [35]:
cal = statcast_batter(start_dt='2025-03-26', end_dt='2025-06-08', player_id=668804)
cal = cal[cal['events'].notna()]
new_cal = code_barrel(cal)
new_cal = new_cal[~new_cal['events'].isin(['walk','hit_by_pitch','field_error','sac_fly','sac_bunt'])]
new_cal['is_barrel'].sum() / new_cal.shape[0]

Gathering Player Data


np.float64(0.08641975308641975)

In [36]:
new_cal['is_barrel'].sum()

np.int64(21)

In [37]:
new_cal.shape[0]

243

In [38]:
park_factors = pd.read_excel('Park Factors.xlsx')
park_factors

,Team,Park,LF_Distance,CF_Distance,RF_Distance,Wall_Height_Min,Wall_Height_Max,HR_LF,Triples_LF,Doubles_LF,Singles_LF,AVG_LF,OBP_LF,SLG_LF,wOBA_LF,R_LF,HR_RF,Triples_RF,Doubles_RF,Singles_RF,AVG_RF,OBP_RF,SLG_RF,wOBA_RF,R_RF
0,Los Angeles Angels,Angel Stadium,330,400,330,5.0,8,1.02,0.95,1.02,0.96,0.98,1.00,1.00,1.00,1.00,1.29,0.55,0.91,0.95,0.98,0.98,1.03,1.00,1.07
1,St. Louis Cardinals,Busch Stadium,336,400,335,8.0,8,0.84,1.10,0.89,1.05,0.98,0.98,0.94,0.96,0.93,0.92,0.75,0.89,1.01,0.96,0.95,0.93,0.95,0.91
2,Arizona Diamondbacks,Chase Field,330,407,335,7.5,25,0.87,1.52,0.95,0.99,0.97,1.00,0.95,0.98,0.95,0.97,2.39,1.01,1.05,1.05,1.03,1.05,1.03,1.09
3,New York Mets,Citi Field,335,408,330,8.0,8,1.07,0.70,0.88,0.86,0.89,0.92,0.93,0.93,0.89,0.98,0.62,0.74,1.01,0.93,0.95,0.91,0.93,0.86
4,Philadelphia Phillies,Citizens Bank Park,329,401,330,6.0,13,1.22,0.99,0.88,0.98,0.98,0.98,1.02,1.00,1.06,1.17,1.10,0.98,0.97,1.01,1.01,1.05,1.03,0.98
5,Detroit Tigers,Comerica Park,345,420,330,7.0,9,0.97,1.85,1.09,1.06,1.06,1.02,1.05,1.03,1.03,0.88,1.69,0.83,0.98,0.94,0.95,0.93,0.94,0.89
6,Colorado Rockies,Coors Field,347,415,350,8.0,17,1.21,2.17,1.43,1.19,1.25,1.16,1.28,1.21,1.27,1.22,1.91,1.12,1.15,1.17,1.10,1.19,1.14,1.33
7,Los Angeles Dodgers,Dodger Stadium,330,395,330,4.5,8,1.21,0.50,0.92,0.99,0.98,0.95,1.01,0.98,0.92,1.04,0.24,1.06,0.96,0.96,0.95,0.96,0.96,0.92
8,Boston Red Sox,Fenway Park,310,390,302,3.0,37,0.97,1.21,1.25,0.99,1.04,1.02,1.05,1.03,1.12,0.82,1.19,1.59,0.97,1.06,1.04,1.05,1.04,0.97
9,Texas Rangers,Globe Life Field,332,400,325,8.0,14,0.96,0.98,0.96,1.00,0.99,1.01,0.98,1.00,0.97,0.95,1.01,1.01,1.04,1.02,1.03,1.00,1.01,0.98


In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


url = "https://www.rotowire.com/baseball/daily-lineups.php"
soup = BeautifulSoup(requests.get(url).content, "html.parser")

data_pitching = []
data_batter = []
team_type = ''

for e in soup.select('.lineup__box ul li'):
    if team_type != e.parent.get('class')[-1]:
        order_count = 1
        team_type = e.parent.get('class')[-1]

    if e.get('class') and 'lineup__player-highlight' in e.get('class'):
        data_pitching.append({
            'date': e.find_previous('main').get('data-gamedate'),
            'game_time': e.find_previous('div', attrs={'class':'lineup__time'}).get_text(strip=True),
            'pitcher_name':e.a.get_text(strip=True),
            'team':e.find_previous('div', attrs={'class':team_type}).next.strip(),
            'lineup_throws':e.span.get_text(strip=True)
        })
    elif e.get('class') and 'lineup__player' in e.get('class'):
        data_batter.append({
            'date': e.find_previous('main').get('data-gamedate'),
            'game_time': e.find_previous('div', attrs={'class':'lineup__time'}).get_text(strip=True),
            'pitcher_name':e.a.get_text(strip=True),
            'team':e.find_previous('div', attrs={'class':team_type}).next.strip(),
            'pos': e.div.get_text(strip=True),
            'batting_order':order_count,
            'lineup_bats':e.span.get_text(strip=True)
        })
        order_count+=1

df_pitching = pd.DataFrame(data_pitching)
df_batter = pd.DataFrame(data_batter)

In [6]:
df_batter

,date,game_time,pitcher_name,team,pos,batting_order,lineup_bats
0,2025-06-03,6:40 PM ET,Jeremy Pena,Astros,SS,1,R
1,2025-06-03,6:40 PM ET,I. Paredes,Astros,3B,2,R
2,2025-06-03,6:40 PM ET,Jose Altuve,Astros,2B,3,R
3,2025-06-03,6:40 PM ET,C. Walker,Astros,1B,4,R
4,2025-06-03,6:40 PM ET,Yainer Diaz,Astros,DH,5,R
...,...,...,...,...,...,...,...
265,2025-06-03,10:10 PM ET,Will Smith,Dodgers,C,5,R
266,2025-06-03,10:10 PM ET,Max Muncy,Dodgers,3B,6,L
267,2025-06-03,10:10 PM ET,Andy Pages,Dodgers,CF,7,R
268,2025-06-03,10:10 PM ET,M. Conforto,Dodgers,LF,8,L


In [7]:
df_batter['team_id'] = df_batter['team'].map(get_team_mapping())

In [8]:
team_ids = df_batter['team_id'].unique()
team_ids = [int(team_id) for team_id in team_ids]
print(team_ids)

[117, 134, 115, 146, 112, 120, 114, 147, 143, 141, 108, 111, 158, 113, 109, 144, 140, 139, 116, 145, 118, 138, 110, 136, 135, 137, 142, 133, 121, 119]


In [79]:
all_player_ids = map_player_ids(team_ids)

No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar

                   name  player_id
0         Alec Burleson     676475
1        Andre Pallante     669467
2       Brendan Donovan     680977
3           Erick Fedde     607200
4          Iván Herrera     671056
..                  ...        ...
772         Tommy Edman     669242
773       Tony Gonsolin     664062
774          Will Klein     694361
775          Will Smith     519293
776  Yoshinobu Yamamoto     808967

[777 rows x 2 columns]


In [17]:
pd.set_option('display.max_columns', None)
start_date = '2025-05-03'
end_date = '2025-06-03'
outcomes = statcast_pitcher(start_dt=start_date, end_dt=end_date, player_id=677958)
# outcomes = outcomes[outcomes['description'].isin(['hit_into_play'])]
outcomes

Gathering Player Data


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches


In [19]:
player_id = playerid_lookup('Rocker', 'Kumar', fuzzy=True)
player_id

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,rocker,kumar,677958,rockk001,rockeku01,-1,2024.0,2025.0


In [ ]:
edman = statcast_batter(start_dt='2025-05-15', end_dt='2025-05-30', player_id=669242)
mapping = {'single': 1, 'double': 2, 'triple': 3, 'home_run': 4}
edman['bases'] = edman['events'].map(lambda x: mapping.get(x, 0))
edman.groupby('game_date')['bases'].sum().reset_index()

Gathering Player Data


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches
0,CH,2025-05-30,87.2,1.86,5.89,"Edman, Tommy",669242,608331,strikeout,swinging_strike,NaN,NaN,NaN,NaN,14,Tommy Edman strikes out swinging.,R,R,L,LAD,NYY,S,2.0,NaN,0,2,2025,1.07,0.36,1.55,2.05,NaN,NaN,NaN,2,4,Bot,NaN,NaN,NaN,NaN,NaN,NaN,-2.963713,-127.007194,-3.877253,12.224883,24.095962,-27.746287,3.37,1.48,NaN,NaN,NaN,87.8,1559,6.5,777708,669224,502671,678391,672724,683011,641355,663757,592450,54.03,NaN,0.000,0.00,1.0,0.0,0.0,NaN,36,4,Changeup,2,5,2,5,5,2,2,5,Standard,Standard,137,-0.009,-0.157,67.5,7.7,NaN,0.157,NaN,-3,-3,0.163,0.163,31,30,31,30,2,1,6.0,2,NaN,1,2.62,1.07,-1.07,43.1,10.269307,0.231303,20.292926,57.423626,26.479223
1,CU,2025-05-30,77.5,1.57,6.13,"Edman, Tommy",669242,608331,NaN,foul,NaN,NaN,NaN,NaN,8,Tommy Edman strikes out swinging.,R,R,L,LAD,NYY,S,NaN,NaN,0,2,2025,-0.79,-1.43,0.24,1.48,NaN,NaN,NaN,2,4,Bot,NaN,NaN,NaN,NaN,NaN,NaN,-1.385508,-112.814177,-0.290748,-6.432368,23.008215,-44.496663,3.37,1.48,3.0,80.7,-28.0,77.3,3000,6.4,777708,669224,502671,678391,672724,683011,641355,663757,592450,54.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36,3,Curveball,2,5,2,5,5,2,2,5,Standard,Standard,328,0.000,0.000,70.8,8.2,NaN,0.000,88.0,-3,-3,0.163,0.163,31,30,31,30,2,1,6.0,2,NaN,1,5.26,-0.79,0.79,50.7,22.141098,-23.118673,27.646621,41.808384,43.036567
2,CH,2025-05-30,85.7,2.03,5.76,"Edman, Tommy",669242,608331,NaN,called_strike,NaN,NaN,NaN,NaN,5,Tommy Edman strikes out swinging.,R,R,L,LAD,NYY,S,NaN,NaN,0,1,2025,0.78,0.65,-0.09,2.39,NaN,NaN,NaN,2,4,Bot,NaN,NaN,NaN,NaN,NaN,NaN,-6.615524,-124.749474,-3.052713,9.417115,23.420273,-24.952485,3.37,1.54,NaN,NaN,NaN,86.3,1488,6.5,777708,669224,502671,678391,672724,683011,641355,663757,592450,54.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36,2,Changeup,2,5,2,5,5,2,2,5,Standard,Standard,129,0.000,-0.058,NaN,NaN,NaN,0.058,NaN,-3,-3,0.163,0.163,31,30,31,30,2,1,6.0,2,NaN,1,2.43,0.78,-0.78,41.4,NaN,NaN,NaN,NaN,NaN
3,FF,2025-05-30,90.4,1.84,5.87,"Edman, Tommy",669242,608331,NaN,called_strike,NaN,NaN,NaN,NaN,1,Tommy Edman strikes out swinging.,R,R,L,LAD,NYY,S,NaN,NaN,0,0,2025,-0.36,0.69,-0.75,3.04,NaN,NaN,NaN,2,4,Bot,NaN,NaN,NaN,NaN,NaN,NaN,-5.703110,-131.699606,-2.665916,-3.199973,24.103962,-23.750709,3.31,1.49,NaN,NaN,NaN,91.1,2278,6.3,777708,669224,502671,678391,672724,683011,641355,663757,592450,54.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36,1,4-Seam Fastball,2,5,2,5,5,2,2,5,Standard,Standard,165,0.000,-0.041,NaN,NaN,NaN,0.041,NaN,-3,-3,0.163,0.163,31,30,31,30,2,1,6.0,2,NaN,1,